To open this notebook in Colab visit https://goo.gl/magenta/groovae-colab

<img src="https://magenta-staging.tensorflow.org/assets/groovae/score-groove.png" alt="GrooVAE Figure" >

## TODO:
<ol>
  <li><strike>NN to Max</strike></li>
  <li><strike>Python Module Everything</strike></li>
  <li><strike>Osc Integration</strike></li>
  <li>Preserve Microtiming</li>
  <li>Control temperature with OSC</li>
  <li>What are mu, sigma?</li>
  <li>tpq=220</li>
</ol>

In [1]:
import copy
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import numpy as np
np.set_printoptions(suppress=True)

#import tensorflow_datasets as tfds
from magenta.models.music_vae.trained_model import TrainedModel

from groovae_max import *

/home/oguz/anaconda3/envs/cc/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/oguz/anaconda3/envs/cc/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [2]:
## Load MIDI files from GMD with MIDI only (no audio) as a tf.data.Dataset
#dataset_2bar = tfds.as_numpy(tfds.load(
#                name="groove/2bar-midionly",
#                split=tfds.Split.VALIDATION,
#                try_gcs=True))
#
#dev_sequences = [quantize(note_seq.midi_to_note_sequence(features["midi"])) for features in dataset_2bar]
#_ = [set_to_drums(s) for s in dev_sequences]
#dev_sequences = [s for s in dev_sequences if is_4_4(s) and len(s.notes) > 0 and s.notes[-1].quantized_end_step > note_seq.steps_per_bar_in_quantized_sequence(s)]
#print(len(dev_sequences))

# Tap2Drum: Generate a beat from any rhythm 

While the Groove model works by removing the micro-timing and velocity information and learning to predict them from just the drum pattern, we can also go in the opposite direction.  Here, we take a representation of a Groove as input (in the form of a rhythm that can have precise timing but where drum categories are ignored) - and then generate drum beats that match the groove implied by this rhythm.  We trained this model by collapsing all drum hits from each beat in the training data to a single "tapped" rhythm, and then learning to decode full beats from that rhythm.  This allows us to input any rhythm we like through the precise onset timings in a "tap" and let the model decode our rhythm into a beat. We can even simply record taps as audio, or extract them from a recording of another instrument, rather than needing a midi controller.

Here are a couple of examples using MIDI rhythms:

In [3]:
groovae_2bar_tap = TrainedModel(config=configs.CONFIG_MAP['groovae_2bar_tap_fixed_velocity'],
                                batch_size=1,
                                checkpoint_dir_or_path=GROOVAE_2BAR_TAP_FIXED_VELOCITY)

#sequence_indices = [1111, 366]
#for i in sequence_indices:
#    s = start_notes_at_0(dev_sequences[i])
#    s = change_tempo(get_tapped_2bar(s, velocity=85, ride=True), dev_sequences[i].tempos[0].qpm)
#    h = change_tempo(drumify(s, groovae_2bar_tap), s.tempos[0].qpm)

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, GrooveLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 256, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 1, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [256, 256], 'enc_rnn_size': [512], 'dropout_keep_prob': 0.3, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [512]

INFO:tensorflow:
Decoder Cells:
  units: [256, 256]



/home/oguz/anaconda3/envs/cc/lib/python3.7/site-packages/magenta/models/music_vae/lstm_utils.py:99: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  name=name),
/home/oguz/anaconda3/envs/cc/lib/python3.7/site-packages/keras/legacy_tf_layers/core.py:261: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/home/oguz/anaconda3/envs/cc/lib/python3.7/site-packages/magenta/contrib/rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._names["W"], [input_size + self._num_units, self._num_units * 4])
/home/oguz/anaconda3/envs/cc/lib/python3.7/site-packages/magenta/contrib/rnn.py:754: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method inst

Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


2022-03-16 13:44:54.209109: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 13:44:54.217790: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-03-16 13:44:54.217814: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Instructions for updating:
Do not call `graph_parents`.
INFO:tensorflow:Unbundling checkpoint.
INFO:tensorflow:Restoring parameters from /tmp/tmpywf7hcoz/groovae_2bar_tap_fixed_velocity/model.ckpt-3668


/home/oguz/anaconda3/envs/cc/lib/python3.7/site-packages/magenta/models/music_vae/base_model.py:199: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))
/home/oguz/anaconda3/envs/cc/lib/python3.7/site-packages/magenta/models/music_vae/base_model.py:205: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))
2022-03-16 13:44:54.498714: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# IO Communication

quantized_start_step, quantized_end_step  
control_changes {
  control_number: 4
  control_value: 90
  is_drum: true
}

In [4]:
max_lst="0. 0.03 120. 0.0625 0.0925 120. 0.125 0.155 120. 0.1875 0.2175 120. 0.25 0.28 120. 0.3125 0.3425 120. 0.375 0.405 0. 0.4375 0.4675 0. 0.5 0.53 0. 0.5625 0.5925 0. 0.625 0.655 0. 0.6875 0.7175 120. 0.75 0.78 0. 0.8125 0.8425 0. 0.875 0.905 120. 0.9375 0.9675 0. 1. 1.03 120. 1.0625 1.0925 120. 1.125 1.155 0. 1.1875 1.2175 0. 1.25 1.28 0. 1.3125 1.3425 0. 1.375 1.405 0. 1.4375 1.4675 0. 1.5 1.53 0. 1.5625 1.5925 0. 1.625 1.655 0. 1.6875 1.7175 0. 1.75 1.78 120. 1.8125 1.8425 120. 1.875 1.905 120. 1.9375 1.9675 120.".split(' ')

BPM=120
# Convert max list to a readable format
midi_array=max_list_to_midi_array(max_lst, BPM)
len(midi_array)
# Convert it into the pre-NN input format
note_sequence=make_tap_sequence(midi_array, BPM)
note_sequence=quantize(note_sequence)
set_to_drums(note_sequence)
# Convert to NN input format
s=copy.deepcopy(note_sequence)
s=start_notes_at_0(s)
s=change_tempo(get_tapped_2bar(s, velocity=VELOCITY, ride=True), BPM)
assert BPM==s.tempos[0].qpm, 'Tempo conversion failed at tapped bar creation'
# Get NN prediction
h=change_tempo(drumify(s, groovae_2bar_tap), BPM)
assert BPM==h.tempos[0].qpm, 'Tempo conversion failed at NN creation'
# Convert to Max array
MAX_array=NN_output_to_Max(h, BPM, beat_quantization_division=64)

In [5]:
MAX_array

array([[ 0.      ,  0.25    , 18.      , 36.      ],
       [ 0.015625,  0.25    , 24.      , 42.      ],
       [ 0.25    ,  0.25    , 28.      , 42.      ],
       [ 0.25    ,  0.25    ,  0.      , 49.      ],
       [ 0.484375,  0.25    , 17.      , 36.      ],
       [ 0.484375,  0.25    , 11.      , 38.      ],
       [ 0.71875 ,  0.25    , 50.      , 42.      ],
       [ 0.984375,  0.25    , 15.      , 38.      ],
       [ 1.      ,  0.25    , 34.      , 42.      ],
       [ 1.234375,  0.25    , 14.      , 38.      ],
       [ 1.5     ,  0.25    , 12.      , 38.      ],
       [ 1.984375,  0.25    , 35.      , 38.      ],
       [ 2.25    ,  0.25    , 27.      , 38.      ],
       [ 2.734375,  0.25    ,  1.      , 50.      ],
       [ 2.984375,  0.25    , 25.      , 38.      ],
       [ 3.703125,  0.25    , 18.      , 38.      ],
       [ 3.96875 ,  0.25    , 45.      , 36.      ],
       [ 3.984375,  0.25    , 54.      , 49.      ],
       [ 4.984375,  0.25    , 58.      , 42.  